In [17]:
from pendulum import spherical_pendulum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# parameters for spherical pendulum
azimuthal_angle = [0]
polar_angle = [math.pi/2]
length, gravity, mass = 1, 10, 1
time_step, time_max = 0.01, 100

# collect data
azimuthal_angle, polar_angle = spherical_pendulum(azimuthal_angle, polar_angle, length, gravity, mass, time_step, time_max)

data = polar_angle
train = data[:8000]
test = data[8001:]

# Autoregression

In [48]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [50]:
'''
Using autoregression, moving average, autoregressive moving average, autoregressive integrated moving average, seasonal
autoregressive integrated moving average
'''

models = [
            AutoReg(train, lags=1), 
            ARIMA(train, order=(0, 0, 1)), 
            ARIMA(train, order=(2,0,1)), 
            ARIMA(train, order=(1,1,1)),
            SARIMAX(train, order=(1,1,1), seasonal_order=(0,0,0,0))
        ]
root_mean_squared_errors = []

for model in models:
    model_fit = model.fit()
    preds = model_fit.predict(start=len(train), end=len(train) + len(test) -1)
    root_mean_squared_errors.append(math.sqrt(mean_squared_error(preds, test)))

print (root_mean_squared_errors)

/home/eric/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/ar_model.py:248: FutureWarning: The parameter names will change after 0.12 is released. Set old_names to False to use the new names now. Set old_names to True to use the old names. 
  warnings.warn(
/home/eric/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/eric/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


[1.9144504611818687, 1.914370223458114, 1.9144642584189149, 1.912800347294748, 1.912800347294748]
